In [3]:
from stable_baselines3 import DQN
import random

/opt/conda/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import gym
import numpy as np
import gymnasium
import tensorflow as tf
import keras
from tensorflow.keras import layers
import time 

In [2]:
!pip install numpy scikit-learn pillow
!pip install stable-baselines3
!pip install shimmy[gym-v21]
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 24.9 MB/s eta 0:00:0000:01
  Attempting uninstall: gymnasium
    Found existing installation: Gymnasium 0.26.3
    Uninstalling Gymnasium-0.26.3:
      Successfully uninstalled Gymnasium-0.26.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.5/734.5 kB 13.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.0 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.25.2-py3-none-any.whl size=852289 sha256=6c4c4ab40c6629d808bb0566d9a3e7e7f5019eba3a7a5170d86cee1f9b074c15
  Stored in directory: /root/.cache/pip/wheels/78/95/2c/ee47a8d43fda6a851e340e77e27cf75b49ff4ce2d1540c0e80
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.26.2
    U

In [3]:
# import time

# import random
# import numpy as np

# from tensorflow import keras
# from gym import spaces
# import gymnasium as gym
# import time

# import os  
# from PIL import Image

# from stable_baselines3 import PPO
# from stable_baselines3.common.env_util import make_vec_env
# from tensorflow.keras import layers
# from stable_baselines3.common.monitor import Monitor

# from stable_baselines3.common.logger import configure

# from stable_baselines3.common.vec_env import DummyVecEnv
# from stable_baselines3.common.monitor import Monitor

In [4]:
# # Define the checkpoint callback
# from stable_baselines3.common.callbacks import CheckpointCallback
# checkpoint_callback = CheckpointCallback(save_freq=10000, save_path='/kaggle/working/', name_prefix='ppo_TX1')

# Importing Data

In [5]:
from PIL import Image
import os
import cv2 


directory = "/kaggle/input/traffic-sign-dataset-classification/traffic_Data/DATA"
def load_images_from_directory(directory):
    X = []  # List to store images
    y = []  # List to store labels

    for class_label in os.listdir(directory):
        class_dir = os.path.join(directory, class_label)
        if os.path.isdir(class_dir):
            for image_file in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_file)
                if image_path.endswith(".jpg") or image_path.endswith(".png"): 
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    image = cv2.resize(image, (28,28))
                    X.append(np.array(image))
                    y.append(class_label)

    return np.array(X), y

X, y = load_images_from_directory(directory)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Model / data parameters
num_classes = 58
input_shape = (28, 28, 1)


# Scale images to the [0, 1] range
x_train = X_train / 255
x_test = X_test / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


x_train shape: (3336, 28, 28, 1)
3336 train samples
834 test samples


In [8]:
from keras.utils import to_categorical

# One-hot encode the labels
y_train_encoded = to_categorical(y_train, num_classes=58)
y_test_encoded = to_categorical(y_test, num_classes=58)

# Standard NN model

In [9]:
def keras_train(batch_size=32, epochs=25):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(num_classes, activation="softmax")
        ]
    )

    model.summary()

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    start_time = time.time()
    model.fit(x_train, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    end_time = time.time()

    score = model.evaluate(x_test, y_test_encoded, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    print("Training Time:", end_time - start_time)
    
keras_train()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 58)                1

# DQN

In [10]:
class DQNEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        super().__init__()

        self.action_space = gym.spaces.Discrete(58)
        self.observation_space = gym.spaces.Box(low=0, high=1,
                                                shape=(28, 28, 1),
                                                dtype=np.float32)

        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0

    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def reset(self):
        self.step_count = 0
        self._seed =None
        obs = self._next_obs()
        return obs

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]

        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs


In [11]:
def TX1_dqn():
    #logger.configure(dir='./logs/TX1_dqn', format_strs=['stdout', 'tensorboard'])
    env = DQNEnv(images_per_episode=1)
    #env = bench.Monitor(env, logger.get_dir())

    model = DQN("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=10000, log_interval=4)
    model.save('dqn_mnist.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = TX1_dqn()
print("DQN Training Time:", time.time() - start_time)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1500     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4        |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.992    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1560     |
|    time_elapsed     | 0        |
|    total_timesteps  | 8        |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0     

/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0.01     |
|    exploration_rate | 0.776    |
| time/               |          |
|    episodes         | 236      |
|    fps              | 1663     |
|    time_elapsed     | 0        |
|    total_timesteps  | 236      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0.02     |
|    exploration_rate | 0.772    |
| time/               |          |
|    episodes         | 240      |
|    fps              | 1663     |
|    time_elapsed     | 0        |
|    total_timesteps  | 240      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0.02     |
|    exploration_rate | 0.768    |
| time/               |          |
|    episodes       

# Evaluation

In [12]:
def dqn_eval(dqn_model, env):
    attempts, correct = 0, 0
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                action, _ = dqn_model.predict(obs)  
                obs, rew, done, _ = env.step(action)
                
                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('Validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) ))

env = DQNEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)


dqn_eval(dqn_model, env)


Validation done...
Accuracy: 0.05048076923076923%
